In [1]:
from dj_notebook import activate

plus = activate()

The .env file has been loaded. See base.py for more information

Output()

/Users/jochen/.virtualenvs/pp/lib/python3.12/site-packages/django/db/backends/utils.py:98: RuntimeWarning: 
Accessing the database during app initialization is discouraged. To fix this warning, avoid executing queries in 
AppConfig.ready() or when your app modules are imported.
  warnings.warn(self.APPS_NOT_READY_WARNING_MSG, category=RuntimeWarning)

In [2]:
import sqlparse

from django.urls import reverse
from django.db import connection

from wagtail.models import Page
from wagtail.api.v2.filters import FieldsFilter
from rest_framework.filters import BaseFilterBackend
from rest_framework.test import APIRequestFactory

from cast.api.views import FilteredPagesAPIViewSet

In [3]:
def format_sql(query):
    return sqlparse.format(query, reindent=True, keyword_case='upper')

def get_formatted_sql(queryset):
    # Get the raw SQL query from the queryset
    sql, params = queryset.query.sql_with_params()
    
    # Format the SQL query
    formatted_sql = format_sql(sql % params)
    
    return formatted_sql

In [4]:
class FieldsFilter(BaseFilterBackend):
    def filter_queryset(self, request, queryset, view):
        """
        This performs field level filtering on the result set
        Eg: ?title=James Joyce
        """
        fields = set(view.get_available_fields(queryset.model, db_fields_only=True))

        # Locale is a database field, but we provide a separate filter for it
        if "locale" in fields:
            fields.remove("locale")

        for field_name, value in request.GET.items():
            if field_name in fields:
                try:
                    field = queryset.model._meta.get_field(field_name)
                except LookupError:
                    field = None

                # Convert value into python
                try:
                    if isinstance(
                        field, (models.BooleanField, models.NullBooleanField)
                    ):
                        value = parse_boolean(value)
                    elif isinstance(field, (models.IntegerField, models.AutoField)):
                        value = int(value)
                    elif isinstance(field, models.ForeignKey):
                        value = field.target_field.get_prep_value(value)
                except ValueError as e:
                    raise BadRequestError(
                        "field filter error. '%s' is not a valid value for %s (%s)"
                        % (value, field_name, str(e))
                    )

                if isinstance(field, TaggableManager):
                    for tag in value.split(","):
                        queryset = queryset.filter(**{field_name + "__name": tag})

                    # Stick a message on the queryset to indicate that tag filtering has been performed
                    # This will let the do_search method know that it must raise an error as searching
                    # and tag filtering at the same time is not supported
                    queryset._filtered_by_tag = True
                else:
                    queryset = queryset.filter(**{field_name: value})

        return queryset

In [5]:
blog = plus.Blog.objects.first()

In [6]:
rf = APIRequestFactory()
pages_endpoint = reverse("cast:api:wagtail:pages:listing")
bad_input = "1%00%EF%BF%BD%EF%BF%BD%EF%BF%BD%EF%BF%BD%252527%252522"
# bad_input = "devops-redux"
# url = f"{pages_endpoint}?child_of={blog.pk}&slug={bad_input}"
url = f"{pages_endpoint}?slug={bad_input}"
request = rf.get(url)
queryset = Page.objects.all().live()
viewset = FilteredPagesAPIViewSet()
# queryset = FieldsFilter().filter_queryset(request, queryset, viewset)
queryset = queryset.filter(**{"slug": "1����%2527%2522"})
print(get_formatted_sql(queryset))
queryset.count()
for page in queryset:
    print(page.pk, page.slug)

SELECT "wagtailcore_page"."id",
       "wagtailcore_page"."path",
       "wagtailcore_page"."depth",
       "wagtailcore_page"."numchild",
       "wagtailcore_page"."translation_key",
       "wagtailcore_page"."locale_id",
       "wagtailcore_page"."latest_revision_id",
       "wagtailcore_page"."live",
       "wagtailcore_page"."has_unpublished_changes",
       "wagtailcore_page"."first_published_at",
       "wagtailcore_page"."last_published_at",
       "wagtailcore_page"."live_revision_id",
       "wagtailcore_page"."go_live_at",
       "wagtailcore_page"."expire_at",
       "wagtailcore_page"."expired",
       "wagtailcore_page"."locked",
       "wagtailcore_page"."locked_at",
       "wagtailcore_page"."locked_by_id",
       "wagtailcore_page"."title",
       "wagtailcore_page"."draft_title",
       "wagtailcore_page"."slug",
       "wagtailcore_page"."content_type_id",
       "wagtailcore_page"."url_path",
       "wagtailcore_page"."owner_id",
       "wagtailcore_page"."seo_title"

In [7]:
query = """
SELECT "wagtailcore_page"."id",
       "wagtailcore_page"."path",
       "wagtailcore_page"."depth",
       "wagtailcore_page"."numchild",
       "wagtailcore_page"."translation_key",
       "wagtailcore_page"."locale_id",
       "wagtailcore_page"."latest_revision_id",
       "wagtailcore_page"."live",
       "wagtailcore_page"."has_unpublished_changes",
       "wagtailcore_page"."first_published_at",
       "wagtailcore_page"."last_published_at",
       "wagtailcore_page"."live_revision_id",
       "wagtailcore_page"."go_live_at",
       "wagtailcore_page"."expire_at",
       "wagtailcore_page"."expired",
       "wagtailcore_page"."locked",
       "wagtailcore_page"."locked_at",
       "wagtailcore_page"."locked_by_id",
       "wagtailcore_page"."title",
       "wagtailcore_page"."draft_title",
       "wagtailcore_page"."slug",
       "wagtailcore_page"."content_type_id",
       "wagtailcore_page"."url_path",
       "wagtailcore_page"."owner_id",
       "wagtailcore_page"."seo_title",
       "wagtailcore_page"."show_in_menus",
       "wagtailcore_page"."search_description",
       "wagtailcore_page"."latest_revision_created_at",
       "wagtailcore_page"."alias_of_id"
FROM "wagtailcore_page"
WHERE ("wagtailcore_page"."live"
       AND "wagtailcore_page"."slug" = 1����%2527%2522)
ORDER BY "wagtailcore_page"."path" ASC
"""
# params = ["devops-redux"]
params = ["1����%2527%2522"]

with connection.cursor() as cursor:
    # cursor.execute(query, params)
    cursor.execute(query)
    results = cursor.fetchall()

for row in results:
    print(row)

ProgrammingError: Müll folgt auf numerische Konstante bei »1�«
LINE 33:        AND "wagtailcore_page"."slug" = 1����%2527%2522)
                                                ^

In [ ]:
for page in plus.Post.objects.all():
    print(page.pk, page.slug)

In [7]:
for field_name, value in request.GET.items():
    print(field_name, value)

slug 1 ����%2527%2522
